In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pyrealsense2 as rs
import cv2

ModuleNotFoundError: No module named 'pyrealsense2'

In [11]:
BAG_FILE_          = "../data/scan4.bag"
CLIPPING_DISTANCE  = 0.25
CLIPPING_TOLERANCE = 0.02 
SURFACE_RATIO      = 0.75
FRAME_WIDTH        = 640
FRAME_HEIGHT       = 480

In [12]:
# Create a realsense pipeline
pipeline = rs.pipeline()

# Create config object
config   = rs.config()

# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, BAG_FILE_)

# Depth stream config
config.enable_stream(rs.stream.depth, FRAME_WIDTH, FRAME_HEIGHT, rs.format.z16, 15)
# RGB stream config
config.enable_stream(rs.stream.color, FRAME_WIDTH, FRAME_HEIGHT, rs.format.rgb8, 15)

# Start streaming from file
profile = pipeline.start(config)

depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to  = rs.stream.color
align     = rs.align(align_to)
colorizer = rs.colorizer()

RuntimeError: Failed to resolve request. Request to enable_device_from_file("../data/scan4") was invalid, Reason: Failed to create ros reader: Error opening file: ../data/scan4

In [5]:
def get_cv_frames (rs_frames):
    # Get depth and color frame
    depth_frame = rs_frames.get_depth_frame()
    color_frame = rs_frames.get_color_frame()

    # Convert depth frame to a metric depth map 
    depth_map = cv2.rotate(np.asanyarray(depth_frame.get_data()), cv2.ROTATE_180) * depth_scale
    depth_map = np.where(depth_map == 0, CLIPPING_DISTANCE * 2, depth_map)
    
    # Convert depth_frame to numpy array to render image in opencv. Used for debug purposes
    depth_color_frame = colorizer.colorize(depth_frame)
    depth_color_img   = cv2.rotate(np.asanyarray(depth_color_frame.get_data()), cv2.ROTATE_180) 
    
    # Convert color_frame to numpy array to render image in opencv
    bgr_color_img = cv2.rotate(np.asanyarray(color_frame.get_data()), cv2.ROTATE_180)
    rgb_color_img = cv2.cvtColor(bgr_color_img, cv2.COLOR_BGR2RGB) 
    return rgb_color_img, depth_map, depth_color_img

In [6]:
cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("Color Stream", cv2.WINDOW_AUTOSIZE)

while True:
    # Get frameset of depth
    frames = pipeline.wait_for_frames()

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    cv_frames = get_cv_frames(aligned_frames)
    
    cv2.imshow("Depth Stream", cv_frames[0])
    cv2.imshow("Color Stream", cv_frames[2])
    key = cv2.waitKey(1)
    # if pressed escape exit program
    if key == 27:
        cv2.destroyAllWindows()
        break

In [7]:
cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow("Raw Depth Stream", cv2.WINDOW_AUTOSIZE)
# cv2.namedWindow("Depth Map", cv2.WINDOW_AUTOSIZE)
while True:
    # Get frameset of depth
    frames = pipeline.wait_for_frames()

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    _, depth_map, _ = get_cv_frames(aligned_frames)

    median = cv2.medianBlur(depth_map, 25)

    cv2.imshow("Raw Depth Stream", depth_map)
    #cv2.imshow("Depth Stream", gray_depth_frame)
    key = cv2.waitKey(1)
    # if pressed escape exit program
    if key == 27:
        cv2.destroyAllWindows()
        break

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/median_blur.simd.hpp:880: error: (-215:Assertion failed) src.depth() == CV_8U && (cn == 1 || cn == 3 || cn == 4) in function 'medianBlur'


In [8]:
import time
import random

from utils.roi import ROIWindow

cv2.namedWindow("Detection Stream", cv2.WINDOW_KEEPRATIO)

roi_tool_w = ROIWindow(
    frame_h = FRAME_HEIGHT, 
    frame_w = FRAME_WIDTH, 
    win_w   = 5,
    win_h   = 25
)

roi_circles_w = ROIWindow(
    frame_h = FRAME_HEIGHT,
    frame_w = FRAME_WIDTH,
    win_w   = 60,
    win_h   = 200
)


roi_tool_w.calibrate_offset_y(40)

roi_circles_w.calibrate_offset_y(40)
roi_circles_w.set_color((0, 255, 0))
tool_idx = 0

while True:
    # Get frameset of depth
    frames = pipeline.wait_for_frames()

    start = time.time()

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    color_frame, depth_map, _ = get_cv_frames(aligned_frames)
    debug_color_frame         = color_frame.copy()
    
    depth_roi = roi_tool_w.get_window(depth_map)
    color_roi = roi_tool_w.get_window(color_frame)

    roi_tool_w.apply_window(debug_color_frame)
    roi_tool_w.apply_hr_calib_line(debug_color_frame, 75, from_bottom = True)
    roi_tool_w.apply_hr_calib_line(debug_color_frame, 15)

    depth_mask = np.where(depth_roi < CLIPPING_DISTANCE + CLIPPING_TOLERANCE, 1, 0).astype(np.uint8)
    
    # avg_depth = np.mean(depth_roi)
    # cv2.putText()

    masked_roi = cv2.bitwise_and(color_roi, color_roi, mask = depth_mask)

    good_surface_ratio = np.sum(depth_mask > 0) / (depth_mask.shape[0] * depth_mask.shape[1])

    ready_4_contour = False
    if (good_surface_ratio >= SURFACE_RATIO):
        # 1. Hough circles method
        roi_circles_w.apply_window(debug_color_frame)
        circles_roi      = roi_circles_w.get_window(color_frame)
        circles_roi_g    = cv2.cvtColor(circles_roi, cv2.COLOR_RGB2GRAY)

        circles          = cv2.HoughCircles(
            circles_roi_g,
            cv2.HOUGH_GRADIENT_ALT,
            1.5,
            20,
            param1    = 275,
            param2    = 0.9,
            minRadius = 12,
            maxRadius = 90
        )
        
        if circles is not None:

            circles = np.uint16(np.around(circles))
            centered = False
            depth_mask_roi    = roi_circles_w.get_window(depth_map)

            for circle in circles[0,:]:
                x, y, radius = circle
                # draw the outer circle
                cv2.circle(circles_roi,(x, y), radius,(0,255,0), 2)
                cv2.circle(depth_mask_roi, (x, y), radius, 1, -1)
                
                # draw the center of the circle
                cv2.circle(circles_roi,(x, y), 2, (0,0,255), 3)

                centered = abs(x - circles_roi.shape[1] / 2) < 10

            if centered:
                tool_idx += 1
                np.save("tool" + str(tool_idx) + ".npy", color_frame)
                
                cv2.circle(circles_roi,(x, y), radius,(0,255,0), 2)
                pts = []
                while(len(pts) < 30):
                    x_p = random.randrange(0, depth_mask_roi.shape[0] - 1)
                    y_p = random.randrange(0, depth_mask_roi.shape[1] - 1)
                    if (depth_mask_roi[x_p, y_p] <= CLIPPING_DISTANCE + CLIPPING_TOLERANCE):
                        pts.append([y_p, x_p])

                for i in range(0, 30):
                    cv2.circle(circles_roi, pts[i], 2, (255, 0, 0), 1)

            roi_circles_w.apply_window_patch(debug_color_frame, circles_roi)

    elapsed = time.time() - start
    cv2.putText(debug_color_frame, "FPS: " + str(round(1 / elapsed, 3)) , (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    

    cv2.imshow("Detection Stream", debug_color_frame)
    key = cv2.waitKey(1)
    # if pressed escape exit program
    if key == 27:
        cv2.destroyAllWindows()
        break